# Long Short Time Memory

Table of contents
1. [Library imports](#Library-imports)
2. [Data Reading](#Data-reading)
    1. [Data cleaning](#Data-cleaning)
    2. [Data normalizarion](#Data-normalization)
3. [LSTM](#LSTM-models-fitting)
    1. [Fine tuning](#Fine-tuning)
    2. [Final model](#Final-model)
    
## Library imports

In [1]:
!pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-1.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached tensorflow-2.8.0-cp310-cp310-manylinux2010_x86_64.whl (497.6 MB)
INFO: pip is looking at multiple versions of pandas to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of numpy to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install -r requirements.txt (line 2), numpy==1.19.5 and tensorflow==2.8.0 because these package versions have con

In [2]:
from utils.utils import mix_data, generate_windows # custom functions
import tensorflow as tf                            # LSTM
import pandas as pd                                # DataFrames
import numpy as np                                 # Vectorial computation
import itertools                                   # Combinatory
import datetime                                    # Dates formatting
# version checking
print(f"""
Tensorflow version -> {tf.__version__}
Pandas version     -> {pd.__version__}
Numpy version      -> {np.__version__}
""")

2022-02-28 10:08:07.938762: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-28 10:08:07.938784: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



Tensorflow version -> 2.8.0
Pandas version     -> 1.4.1
Numpy version      -> 1.22.2



## Data reading

In [3]:
df = pd.read_csv('../data/radon-data.csv')   # CSV reading
df.head()

,id,time,radon,temperature,humidity,pressure,tvoc,sensor_id,state,state_time
0,21906,1569405062,202,25,50,1015,0,2,Off,1569404979
1,21907,1569405663,258,25,51,1015,0,2,On,1569405215
2,21908,1569406264,202,24,51,1015,0,2,Off,1569405671
3,21909,1569406865,182,24,51,1015,0,2,Off,1569406848
4,21910,1569407466,189,24,51,1015,0,2,Off,1569406866


### Data cleaning

In [4]:
# Now we will parse the data, so it is usable
df.time = pd.to_datetime(df['time'], unit='s', origin='unix')      # date parse
df = df.drop(columns = ["time", "id", "sensor_id", "state_time"])  # drop useless columns
df.state = (df.state == "On").astype(int)                          # binarize state
df.head()

,radon,temperature,humidity,pressure,tvoc,state
0,202,25,50,1015,0,0
1,258,25,51,1015,0,1
2,202,24,51,1015,0,0
3,182,24,51,1015,0,0
4,189,24,51,1015,0,0


In [5]:
# we now select the data from the summer (longest clean data)
df_summer = df.iloc[36000:48000]
df_summer.head()

,radon,temperature,humidity,pressure,tvoc,state
36000,388,24,57,1010,4,1
36001,383,24,57,1010,2,1
36002,398,24,57,1010,6,1
36003,388,24,57,1010,8,1
36004,388,24,57,1010,10,1


### Data normalization

In [6]:
summer_min = df_summer.min() # needed to de-normalize data
summer_max = df_summer.max()

df_summer_normalized = (df_summer - summer_min) / (summer_max - summer_min)
df_summer_normalized.head()

,radon,temperature,humidity,pressure,tvoc,state
36000,0.119640,0.714286,0.666667,0.5,0.003463,1.0
36001,0.117911,0.714286,0.666667,0.5,0.001732,1.0
36002,0.123098,0.714286,0.666667,0.5,0.005195,1.0
36003,0.119640,0.714286,0.666667,0.5,0.006926,1.0
36004,0.119640,0.714286,0.666667,0.5,0.008658,1.0


## LSTM models fitting

In [ ]:
# We will use early stopping to select the best model
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)

# percentage of train samples
train_pct = 0.9

# We will store all the results in a data frame
results = pd.DataFrame(
    {i: [None]*15 for i in [1, 5, 10, 15, 25]}
)

cnt = 0 # helps counting

for number_of_covariates in range(4):
    for covariate_combination in itertools.combinations(("state", "humidity", "pressure", "tvoc"), 
                                                        number_of_covariates):
        print(f"EXECUTING OVER: {covariate_combination}".center(100, "*"))
        # We compute for each window size
        for window_size in [1, 5, 10, 15, 25]:
            print("\n")
            print(f"EXECUTING {window_size} WINDOW SIZE".center(100), "-")
            print("\n")
            windows = {i: generate_windows(df_summer_normalized[i], window_size, 1)
                        for i in ["radon"] + list(covariate_combination) }
            
            print("Windows computed, now building the model")
            
            data = mix_data(windows)
            N_total          = data.shape[0]
            test             = data[int(train_pct * N_total):, :, :]
            data             = data[:int(train_pct * N_total), :, :]
            
            model = tf.keras.models.Sequential(
                [
                    tf.keras.layers.InputLayer((window_size, len(windows.keys()))),
                    tf.keras.layers.LSTM(window_size * 2),
                    tf.keras.layers.Dense(16, activation = "relu"),
                    tf.keras.layers.Dense(16, activation = "relu"),
                    tf.keras.layers.Dense(1)
                ],
            )
            
            model.compile(
                loss = tf.keras.losses.MeanSquaredError(),
                metrics = tf.keras.metrics.RootMeanSquaredError(),
                optimizer = tf.keras.optimizers.RMSprop()
            )
            
            model.summary()
            
            model.fit(
                x                = data[:, :window_size, :],
                y                = data[:, window_size:, 0],
                epochs           = 1_000,
                shuffle          = False,
                validation_split = 0.2, 
                callbacks        = [callback,],
                verbose          = 0,
            )
            
            # Model trained
            
            # Now compute the test predictions and add them to the data frame
            rescale    = lambda x: x * (summer_max.radon - summer_min.radon) + summer_min.radon
            prediction = rescale( model.predict(test[:, :window_size, :]) )
            real       = rescale( test[:, window_size:, 0] )
            
            rmse_test = ( sum((prediction - real) ** 2) / len(prediction) ) ** (1/2)
            results[window_size][cnt] = rmse_test 
            print(f"Window {window_size}, covariates {covariate_combination}: {results[window_size][cnt]}")
        cnt += 1    
        print("*" * 100, "\n"*2)

### Fine tuning
Once we know what model behaves the best, we try to explode this configuration.

In [ ]:
# We will use early stopping to select the best model
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)

# percentage of train samples
train_pct = 0.9

# We will store all the results in a data frame
results = pd.DataFrame(
    {i: [None]*4 for i in [4, 8, 16, 32]}
)

# dense configurations:
dense_combination_config = [[16], [16, 16], [32], [32, 32]]

window_size = 15

# we do not need to create the data in a loop:
windows = {i: generate_windows(df_summer_normalized[i], window_size, 1)
                for i in ["radon", "state"] }
data    = mix_data(windows)
N_total = data.shape[0]
test    = data[int(train_pct * N_total):, :, :]
data    = data[:int(train_pct * N_total), :, :]

for idx, dense_config in enumerate(dense_combination_config):
    for num_hidden_units in [4, 8, 16, 32]:
        
        model = tf.keras.models.Sequential(
            [
                tf.keras.layers.InputLayer((window_size, len(windows.keys()))),
                tf.keras.layers.LSTM(num_hidden_units),
                *[tf.keras.layers.Dense(i, activation = "relu") for i in dense_config],
                tf.keras.layers.Dense(1)
            ],
        )

        model.compile(
            loss = tf.keras.losses.MeanSquaredError(),
            metrics = tf.keras.metrics.RootMeanSquaredError(),
            optimizer = tf.keras.optimizers.RMSprop()
        )

        model.summary()

        model.fit(
            x                = data[:, :window_size, :],
            y                = data[:, window_size:, 0],
            epochs           = 1_000,
            shuffle          = False,
            validation_split = 0.2, 
            callbacks        = [callback,],
            verbose          = 0,
        )

        # Model trained

        # Now compute the test predictions and add them to the data frame
        rescale    = lambda x: x * (summer_max.radon - summer_min.radon) + summer_min.radon
        prediction = rescale( model.predict(test[:, :window_size, :]) )
        real       = rescale( test[:, window_size:, 0] )

        rmse_test = ( sum((prediction - real) ** 2) / len(prediction) ) ** (1/2)
        results[num_hidden_units][idx] = rmse_test 
        print(f"Config {idx} with {num_hidden_units} hidden units -> {rmse_test}")
        print("*" * 100, "\n"*2)

## Final model

In [7]:
# We will use early stopping to select the best model
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)

# percentage of train samples
train_pct = 0.9

window_size = 15

# we do not need to create the data in a loop:
windows = {i: generate_windows(df_summer_normalized[i], window_size, 1)
                for i in ["radon", "state"] }
data    = mix_data(windows)
N_total = data.shape[0]
test    = data[int(train_pct * N_total):, :, :]
data    = data[:int(train_pct * N_total), :, :]

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.InputLayer((window_size, len(windows.keys()))),
        tf.keras.layers.LSTM(4),
        tf.keras.layers.Dense(32, activation = "relu"),
        tf.keras.layers.Dense(1)
    ],
)

model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    metrics = tf.keras.metrics.RootMeanSquaredError(),
    optimizer = tf.keras.optimizers.RMSprop()
)

model.summary()

model.fit(
    x                = data[:, :window_size, :],
    y                = data[:, window_size:, 0],
    epochs           = 1_000,
    shuffle          = False,
    validation_split = 0.2, 
    callbacks        = [callback,],
)

# Model trained

# Now compute the test predictions and add them to the data frame
rescale    = lambda x: x * (summer_max.radon - summer_min.radon) + summer_min.radon
prediction = rescale( model.predict(test[:, :window_size, :]) )
real       = rescale( test[:, window_size:, 0] )

rmse_test = ( sum((prediction - real) ** 2) / len(prediction) ) ** (1/2)
print(f"RMSE -> {rmse_test}")

2022-02-28 10:08:25.568842: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-28 10:08:25.568873: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-28 10:08:25.568891: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (satira): /proc/driver/nvidia/version does not exist
2022-02-28 10:08:25.569382: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4)                 112       
                                                                 
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 305
Trainable params: 305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
270/270 [==============================] - 4s 10ms/step - loss: 0.0027 - root_mean_squared_error: 0.0521 - val_loss: 3.8880e-04 - val_root_mean_squared_error: 0.0197
Epoch 2/1000
270/270 [==============================] - 2s 9ms/step - loss: 0.0020 - root_mean_squared_error: 0.0442 - val_loss: 3.2512e-04 - v

270/270 [==============================] - 3s 11ms/step - loss: 9.4188e-05 - root_mean_squared_error: 0.0097 - val_loss: 7.0376e-05 - val_root_mean_squared_error: 0.0084
Epoch 42/1000
270/270 [==============================] - 3s 11ms/step - loss: 8.5433e-05 - root_mean_squared_error: 0.0092 - val_loss: 6.8861e-05 - val_root_mean_squared_error: 0.0083
Epoch 43/1000
270/270 [==============================] - 3s 11ms/step - loss: 8.6146e-05 - root_mean_squared_error: 0.0093 - val_loss: 6.7171e-05 - val_root_mean_squared_error: 0.0082
Epoch 44/1000
270/270 [==============================] - 3s 11ms/step - loss: 8.4518e-05 - root_mean_squared_error: 0.0092 - val_loss: 6.5632e-05 - val_root_mean_squared_error: 0.0081
Epoch 45/1000
270/270 [==============================] - 3s 11ms/step - loss: 8.0763e-05 - root_mean_squared_error: 0.0090 - val_loss: 6.4085e-05 - val_root_mean_squared_error: 0.0080
Epoch 46/1000
270/270 [==============================] - 3s 11ms/step - loss: 8.0129e-05 - roo

Epoch 86/1000
270/270 [==============================] - 3s 10ms/step - loss: 4.4547e-05 - root_mean_squared_error: 0.0067 - val_loss: 3.8943e-05 - val_root_mean_squared_error: 0.0062
Epoch 87/1000
270/270 [==============================] - 3s 11ms/step - loss: 4.4514e-05 - root_mean_squared_error: 0.0067 - val_loss: 3.8683e-05 - val_root_mean_squared_error: 0.0062
Epoch 88/1000
270/270 [==============================] - 3s 10ms/step - loss: 4.4134e-05 - root_mean_squared_error: 0.0066 - val_loss: 3.8882e-05 - val_root_mean_squared_error: 0.0062
Epoch 89/1000
270/270 [==============================] - 3s 11ms/step - loss: 4.3889e-05 - root_mean_squared_error: 0.0066 - val_loss: 3.8991e-05 - val_root_mean_squared_error: 0.0062
Epoch 90/1000
270/270 [==============================] - 3s 11ms/step - loss: 4.3799e-05 - root_mean_squared_error: 0.0066 - val_loss: 3.8827e-05 - val_root_mean_squared_error: 0.0062
Epoch 91/1000
270/270 [==============================] - 3s 9ms/step - loss: 4.3

270/270 [==============================] - 3s 10ms/step - loss: 3.8107e-05 - root_mean_squared_error: 0.0062 - val_loss: 3.8163e-05 - val_root_mean_squared_error: 0.0062
Epoch 131/1000
270/270 [==============================] - 3s 11ms/step - loss: 3.8064e-05 - root_mean_squared_error: 0.0062 - val_loss: 3.8131e-05 - val_root_mean_squared_error: 0.0062
Epoch 132/1000
270/270 [==============================] - 3s 10ms/step - loss: 3.7953e-05 - root_mean_squared_error: 0.0062 - val_loss: 3.8112e-05 - val_root_mean_squared_error: 0.0062
Epoch 133/1000
270/270 [==============================] - 3s 9ms/step - loss: 3.7956e-05 - root_mean_squared_error: 0.0062 - val_loss: 3.8074e-05 - val_root_mean_squared_error: 0.0062
Epoch 134/1000
270/270 [==============================] - 3s 10ms/step - loss: 3.7828e-05 - root_mean_squared_error: 0.0062 - val_loss: 3.8103e-05 - val_root_mean_squared_error: 0.0062
Epoch 135/1000
270/270 [==============================] - 3s 11ms/step - loss: 3.7671e-05 -

270/270 [==============================] - 3s 10ms/step - loss: 3.6414e-05 - root_mean_squared_error: 0.0060 - val_loss: 3.7259e-05 - val_root_mean_squared_error: 0.0061
Epoch 175/1000
270/270 [==============================] - 3s 10ms/step - loss: 3.6368e-05 - root_mean_squared_error: 0.0060 - val_loss: 3.7543e-05 - val_root_mean_squared_error: 0.0061
Epoch 176/1000
270/270 [==============================] - 3s 10ms/step - loss: 3.6396e-05 - root_mean_squared_error: 0.0060 - val_loss: 3.7504e-05 - val_root_mean_squared_error: 0.0061
Epoch 177/1000
270/270 [==============================] - 3s 10ms/step - loss: 3.6386e-05 - root_mean_squared_error: 0.0060 - val_loss: 3.7413e-05 - val_root_mean_squared_error: 0.0061
Epoch 178/1000
270/270 [==============================] - 3s 10ms/step - loss: 3.6405e-05 - root_mean_squared_error: 0.0060 - val_loss: 3.7566e-05 - val_root_mean_squared_error: 0.0061
Epoch 179/1000
270/270 [==============================] - 3s 10ms/step - loss: 3.6326e-05 

In [9]:
model.save("../output/radon.h5", save_format = "h5")

In [10]:
model.save("../output/radon.pb", save_format = "tf")

2022-02-28 10:21:13.375985: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../output/radon.pb/assets


INFO:tensorflow:Assets written to: ../output/radon.pb/assets


In [9]:
summer_min, summer_max

(radon           42
 temperature     19
 humidity        39
 pressure       998
 tvoc             0
 state            0
 dtype: int64,
 radon          2934
 temperature      26
 humidity         66
 pressure       1022
 tvoc           1155
 state             1
 dtype: int64)

In [10]:
test.shape

(1199, 16, 2)

In [ ]:
pd.DataFrame(
    {
        "predictions": prediction.flatten(),
        "real": real.flatten(),
    }
).to_csv("../data/predictions.csv", index = False)